# Replicate the MineRL Colab Example
Get [this notebook](https://colab.research.google.com/drive/1laXCpyf0k6O8Oo1AvUK4UrnywK7IcEh3?usp=sharing#scrollTo=2ZDmSMRP8eto) working but with MineRL v1.0.0; priority is: install, and run.

N.B. I've commented out the env.close() at the end to make this "Run All" safe - but it can be used to safely close a finished env before creating a new one. 

## Setup and Imports

In [1]:
#@title Setup Java and Display modules
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

In [ ]:
#@title Install MineRL and Display Packages
# %%capture
# !pip3 install git+https://github.com/minerllabs/minerl@v1.0.0
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender

In [4]:
#@title Import Packages
# %%capture
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

## Action Generation Functions
Similar to the original doc - generates a long string of jumping, running and
attacking to get some movement through the space

In [5]:
def get_action_sequence(repeats = 10):
    """
    Specify the action sequence for the agent to execute.
    Let's run around with sharp turns
    """
    action_sequence = []
    for _ in range(repeats):
      action_sequence += [''] * 20 # wait 1 sec
      action_sequence += ['forward attack jump'] * 20 # run forward + attack 1 sec
      action_sequence += ['camera:[0,-5]'] * 10 # turn 50deg
      action_sequence += ['forward attack jump'] * 20 # run forward + attack 1 sec
      action_sequence += ['camera:[0,10]'] * 10 # turn back 100deg
      action_sequence += ['forward attack jump'] * 40 # run forward + attack 2 secs
      action_sequence += ['camera:[0,-10]'] * 10 # turn 100deg
      action_sequence += ['forward attack jump'] * 20 # run forward + attack 1 secs
      action_sequence += ['camera:[0,5]'] * 10 # turn back 50deg
    return action_sequence

def str_to_act(env, actions):
    """
    Simplifies specifying actions for the scripted part of the agent.
    Some examples for a string with a single action:
        'craft:planks'
        'camera:[10,0]'
        'attack'
        'jump'
        ''
    There should be no spaces in single actions, as we use spaces to separate actions with multiple "buttons" pressed:
        'attack sprint forward'
        'forward camera:[0,10]'

    :param env: base MineRL environment.
    :param actions: string of actions.
    :return: dict action, compatible with the base MineRL environment.
    """
    act = env.action_space.noop()
    for action in actions.split():
        if ":" in action:
            k, v = action.split(':')
            if k == 'camera':
                act[k] = eval(v)
            else:
                act[k] = v
        else:
            act[action] = 1
    return act

## Setup the Environment
Run once/everytime you run a cleanup

In [6]:
display = Display(visible=0, size=(400, 300))
display.start();

In [7]:
env = gym.make('MineRLBasaltFindCave-v0')
env = Recorder(env, './video', fps=60)

/home/cyprien/anaconda3/envs/mc/lib/python3.9/runpy.py:127: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


## Run a Run in the Environment
(Can be rerun multiple times with different seeds or modifying the get_action_sequence function)

In [8]:
action_sequence = get_action_sequence(repeats = 5)
env.seed(2413)
obs = env.reset()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [9]:
total_reward = 0
for i, action in enumerate(tqdm(action_sequence)):
    obs, reward, done, _ = env.step(str_to_act(env, action))
    total_reward += reward
    if done:
        break

  0%|          | 0/800 [00:00<?, ?it/s]

In [10]:
env.release()
env.play()

print(f'\nTotal reward = {total_reward}')

TypeError: got an unexpected keyword argument 'progress_bar'

## Clean up
If this has been run, you'll need to create the environment again.
Commented for now, but if not be careful with that "Run All" button!

In [ ]:
# env.close()

In [ ]:
!python --version

In [6]:
!python --version

Python 3.8.16
